In [ ]:
#\\Standard libraries
from distutils.command.config import config
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle as pkl
import yaml
import os
from numpy import linalg as LA

import sys
sys.path.append(os.path.dirname(os.getcwd()))
# sys.path.append(os.path.dirname(os.getcwd()) + '/data')
# sys.path.append(os.path.dirname(os.getcwd()) + '/utils')
# sys.path.append(os.path.dirname(os.getcwd()) + '/runners')


#\\Own libraries
from model.ML import *
from data.sample_generator import *
from utils.util import *
from runners.runner_ML import *
from pathlib import Path

Settings

In [ ]:
######################
###  Load parameters of the system ###
######################
dirPath = os.path.dirname(os.getcwd())
with open(dirPath + '/config.yml', 'r') as f:
    aux = yaml.load(f,  Loader=yaml.FullLoader)
config = dict2namespace(aux)



######################
###  General setup ###
######################
SEED = 123
torch.manual_seed(SEED)
useGPU = False # If true, and GPU is available, use it.
loadChannelInput = True
LANGEVIN_DETECTOR = True
CLASSICAL_DETECTORS = True

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda'
else:
    device = 'cpu'



Load data

In [ ]:
if loadChannelInput == True:

    #\\ IID channels
    # with open(dirPath + '/data/Hiid_5000bs_3264', 'rb') as fp:
    #     H = pkl.load(fp)
    # batch_size = H.shape[0]

    #\\ Kronecker channels
    # with open(dirPath + '/data/H_5000bs_3264', 'rb') as fp:
    #     H = pkl.load(fp)
    # batch_size = H.shape[0]

    #\\ 3gpp case
    # mat_contents = sio.loadmat(dirPath + '/data/H_bank.mat')
    # H = mat_contents['H_bank']
    # # H = torch.tensor(H[:, :, 0:config.NT])
    # H = torch.tensor(H[:, :, random.sample(range(100), config.NT)])#Pick up NT random users from 100.
    # batch_size = H.shape[0]
    # Hr = torch.real(H)
    # Hi = torch.imag(H)

    # h1 = torch.cat((Hr, -1. * Hi), dim=2)
    # h2 = torch.cat((Hi, Hr), dim=2)
    # H = torch.cat((h1, h2), dim=1)

    #\\ Real channel
    int_min = 0
    int_max = 200

    H, batch_size = loadRealChannels(dirPath, int_min, int_max)

    #Create generator
    generator = sample_generator(batch_size, config.mod_n, config.NR)
else:
    batch_size = 5000
    generator = sample_generator(batch_size, config.mod_n, config.NR)
    H, y, x, j_indices, noise_sigma = generator.give_batch_data(config.NT, 
                                                                snr_db_min=config.SNR_dBs[config.NT][0], snr_db_max=config.SNR_dBs[config.NT][0], 
                                                                batch_size=batch_size, correlated_flag=config.corr_flag, rho=config.rho)  



Run detectors

In [4]:

serML = runML(config, generator, batch_size, device, H = H)
print(serML)
# with open(dirPath + '/results/ML_results', "wb") as output_file:
#     pkl.dump(serML, output_file)


In [ ]:
with open(dirPath + '/results/ML_results', "wb") as output_file:
    pkl.dump(serML, output_file)
